In [180]:
import json
import pandas as pd
import requests
import pickle

In [181]:
follows = pd.read_csv('../../shared-folder-gald/data/follow-link.csv')

In [182]:
# users = pd.read_csv('../../shared-folder-gald/data/user_info_ALL.p')

with open('../../shared-folder-gald/data/user_info_ALL.p', 'rb') as f:
    users = pickle.load(f)

In [183]:
prev_users = pd.read_csv('../data/liked_videos.csv')

In [184]:
prev_users['username'].nunique()

967

In [185]:
usernames = list(users.keys())

In [186]:
len(usernames)

15268

In [187]:
# Get the set of usernames in prev_users
prev_usernames = set(prev_users['username'])

# Remove existing usernames from the full list
usernames = [user for user in usernames if user not in prev_usernames]

In [188]:
len(usernames)

14301

In [189]:
with open("../../shared-folder-gald/keys1.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [190]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [191]:
def get_liked_info(username, access_token,fields ="id, username" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/liked_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [192]:
access_token = get_access_token(client_key, client_secret)

Ran up until [13013:14014]]

In [193]:
liked_videos = []
for username in usernames[14014:14302]:
    liked_videos.append(get_liked_info(username, access_token))

In [194]:
combined = {}
for i, username in enumerate(usernames[14014:14302]):
    combined[username] = liked_videos[i]

In [195]:
rows = []

for key, value in combined.items():
    if value and 'user_liked_videos' in value:  # Ensure key has data
        for video in value['user_liked_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'id': video['id'],  # Video ID
                'creator': video['username']  # Video username
            })

# Create DataFrame
likes_df = pd.DataFrame(rows)

# Display DataFrame
print(likes_df)
print('Number of unique users:', likes_df['username'].nunique())

             username                   id                creator
0    travelinginstyle  7492894880700583210         notchiaraaceto
1    travelinginstyle  7492864509585968427  ____rewardiffound____
2    travelinginstyle  7474643922929618222            fearlessmco
3    travelinginstyle  7492892755673238826        skotlinmalmrose
4    travelinginstyle  7492907587524939039            roxaannexox
..                ...                  ...                    ...
110     financialeyes  7489948830285221142             jaymeerees
111     financialeyes  7489181937920265494            mantisbeats
112     financialeyes  7488308106288483606            maryamsuuii
113     financialeyes  7489824307871452418            lewisbaaron
114     financialeyes  7492772464531967254            lewisbaaron

[115 rows x 3 columns]
Number of unique users: 6


In [196]:
likes_prev = pd.read_csv("../../shared-folder-gald/data/liked_videos.csv")

In [197]:
likes_df.to_csv("../../shared-folder-gald/data/liked_videos.csv", mode="a", index=False, header=False)

In [198]:
likes = pd.read_csv("../../shared-folder-gald/data/liked_videos.csv")

In [199]:
print('Number of unique users before: ', likes_prev['username'].nunique())
print('Number of unique users after: ', likes['username'].nunique())
print('Number of gathered likes: ', len(likes))

Number of unique users before:  1167
Number of unique users after:  1173
Number of gathered likes:  21208


In [201]:
(likes['username'].nunique()/len(usernames))*100

8.202223620725825